# Shell Meister Exercise

This notebook implements a Shell Command Assistant that:
1. Translates natural language to Linux shell commands
2. Explains what each command does
3. Stores successful commands in memory for future reference

**Note**: This is designed for a Linux devcontainer environment. Use with caution as shell commands can modify your system.

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Initialize Kernel and Services 

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

kernel.add_service(chat_service)

### 2. Setup Memory 

In [ ]:
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

### 3. Define Prompts (experiment with these!)

In [ ]:
# Prompt for translating natural language to shell commands
command_prompt = """
Task: {{$task}}

Convert this task into a Linux shell command. The command will run in a devcontainer environment.
Ensure the command is safe and won't cause system damage.
Use common Linux tools and commands.

Provide the response in this format:
🔧 Command: [the shell command]
📝 Explanation: [explain what each part of the command does]
⚠️ Caution: [any warnings or considerations]
"""

# Prompt for suggesting command variations
variation_prompt = """
Command: {{$command}}

Suggest alternative ways to achieve the same result using different Linux commands or options.
Focus on common alternatives that might be more efficient or easier to understand.

Provide the response in this format:
🔄 Alternative 1:
  Command: [alternative command]
  Why: [why this might be better]

🔄 Alternative 2:
  Command: [alternative command]
  Why: [why this might be better]
"""

# Create semantic functions
command_function = kernel.add_function(
    prompt=command_prompt,
    function_name="translate_to_command",
    plugin_name="ShellMeister",
)

variation_function = kernel.add_function(
    prompt=variation_prompt,
    function_name="suggest_variations",
    plugin_name="ShellMeister",
)

### 4. Example Usage

In [ ]:
# Example tasks to convert to commands
tasks = [
    "Find all Python files in the current directory and count how many lines each has",
    "Show system resource usage, including CPU and memory",
    "Create a backup of all .txt files, compressed into a timestamped archive"
]

for task in tasks:
    # Get command for task
    command_result = await kernel.invoke(
        command_function,
        task=task
    )
    print(f"\nTask: {task}\n{command_result}")
    
    # Get command variations
    variations = await kernel.invoke(
        variation_function,
        command=str(command_result)
    )
    print(f"\nVariations:\n{variations}")
    
    # Store in memory
    await memory.save_information(
        collection="shell_commands",
        id=f"cmd_{hash(task)}",
        text=f"Task: {task}\n\nSolution:\n{command_result}\n\nVariations:\n{variations}"
    )

### 5. Search Command History

In [ ]:
# Search for commands by description or purpose
search_terms = ["file search", "backup", "system monitoring"]

for term in search_terms:
    print(f"\nSearching commands related to '{term}':\n")
    results = await memory.search("shell_commands", term)
    for result in results:
        print(f"Match (relevance: {result.relevance}):\n{result.text}\n")

### 6. Additional exercises

Try these exercises:
1. Modify the command_prompt to include examples of command usage
2. Add a new prompt to explain complex pipeline commands
3. Create a function to rate command complexity and suggest simpler alternatives
4. Implement a way to categorize commands by their purpose (file ops, system monitoring, etc.)
5. Add safety checks to prevent potentially dangerous commands

**Remember**: Always review commands before executing them, especially if they modify the system!